In [ ]:
# import libraries
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score
from xgboost import XGBClassifier
from matplotlib import pyplot
import shap
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, precision_recall_curve, auc


# specifying constant parameters
random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)

# code for uploading pickled data
M_X_train_entFe = pickle.load(open("pure_data\M_X_train_entFe.pkl", 'rb'))
M_Y_train_entFe = pickle.load(open("pure_data\M_Y_train_entFe.pkl", 'rb'))
F_X_train_entFe = pickle.load(open("pure_data\F_X_train_entFe.pkl", 'rb'))
F_Y_train_entFe = pickle.load(open("pure_data\F_Y_train_entFe.pkl", 'rb'))

# M_X_train_entFe = pd.concat(M_X_train_entFe.values(), join='inner', ignore_index=True)
# M_Y_train_entFe = pd.concat(M_Y_train_entFe.values(), join='inner', ignore_index=True)
# F_X_train_entFe = pd.concat(F_X_train_entFe.values(), join='inner', ignore_index=True)
# F_Y_train_entFe = pd.concat(F_Y_train_entFe.values(), join='inner', ignore_index=True)
# select features
M_selected_features = ['Age', 'Relationship with family head','Respondent live in household',
                        'Known HIV status','Bought/sold sex in the past 12 months',
                        'Condom was used at last paid sex in past 12 months','LAg: recent/long term infection',
                        'CD4 category','on ART','ART initiated past 12 months those ever on ART',
                        'last sex partner relations past 12 months','Sick to work last three months',
                        'Ever attended school','Ever enrolled in school','Highest level of education',
                        'Highest grade at that school level', 'Work for payment in last 12 months',
                        'Ever married/lived together','Avoiding pregnancy','CFcumcision status',
                        'Age at fFst sex', 'Ever sought TB treatment','Alcohol drink frequency',
                        'Urban area indicator','Wealth quintile', 'Sexual intercourse in the past 12 months',
                        'Ever had sexual intercourse'] 
  
M_X_train_selected = M_X_train_entFe[M_selected_features]

F_selected_features = ['Age', 'Relationship with family head', 'Respondent live in household',
       'Sick to work last three months', 'Ever attended school',
       'Ever enrolled in school', 'Highest level of education',
       'Highest grade at that school level',
       'Work for payment in last 12 months', 'Ever married/lived together',
       'Number of pregnancies',
       'Ever had a pregnancy that resulted in a live bFth',
       'No. of childen given bFth since 2012', 'Pregnacy status now',
       'Avoiding pregnancy', 'Age at fFst sex', 'Ever sought TB treatment',
       'Alcohol drink frequency', 'Urban area indicator', 'Known HIV status',
       'Wealth quintile', 'tested HIV at pregnancy',
       'Delivered 12 months preceeding survey',
       'Self-reported ARV status during pregnancy', 'Breastfed last child',
       'Sexual intercourse past 12 months', 'Ever had sexual intercourse',
       'Bought/sold sex in the past 12 months',
       'Condom was used at last paid sex in past 12 months',
       'last sex partner relations past 12 months',
       'LAg: recent/long term infection', 'CD4 category', 'On ART',
       'ART initiated past 12 months those ever on ART',
       'Whether ARVs detected']
   
     
F_X_train_selected = F_X_train_entFe[F_selected_features]

#================================================================================================================================
# parameters space, models definition and random grid search
#================================================================================================================================

# stratified k-fold and model initialization
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)
scoring = ['f1', 'recall', 'precision']

# XGBoost space
params_xgb = {'learning_rate': np.linspace(0, 1, 11),
              'min_split_loss': np.linspace(0, 1, 6),
              'max_depth': np.linspace(2, 10, 5, dtype=int),
              'min_child_weight': np.linspace(1, 20, 20, dtype=int),
              'colsample_bytree': np.linspace(0.5, 1, 6),
              'reg_alpha': np.linspace(0, 1, 11),
              'scale_pos_weight': np.linspace(4, 50, 24, dtype=int),
              'n_estimators': np.linspace(10, 450, 12, dtype=int),
              'reg_lambda': np.linspace(0, 10, 11),
             }

M_xgb_selected = RandomizedSearchCV(estimator=XGBClassifier(booster='gbtree',
                                                                     objective='binary:logistic',
                                                                     random_state=random_seed,
                                                                     nthread=1,
                                                                     verbosity=1),
                                             param_distributions=params_xgb,
                                             n_iter=250,
                                             scoring=scoring,
                                             cv=kfold,
                                             n_jobs=-1,
                                             random_state=random_seed,
                                             verbose=2,
                                             refit='f1')

F_xgb_selected = RandomizedSearchCV(estimator=XGBClassifier(booster='gbtree',
                                                                     objective='binary:logistic',
                                                                     random_state=random_seed,
                                                                     nthread=1,
                                                                     verbosity=1),
                                             param_distributions=params_xgb,
                                             n_iter=250,
                                             scoring=scoring,
                                             cv=kfold,
                                             n_jobs=-1,
                                             random_state=random_seed,
                                             verbose=2,
                                             refit='f1')

M_xgb_selected.fit(M_X_train_selected, M_Y_train_entFe)
dump(M_xgb_selected, "xgb_selected\M_xgb_selected_pure_data.joblib")

F_xgb_selected.fit(F_X_train_selected, F_Y_train_entFe)
dump(F_xgb_selected, "xgb_selected\F_xgb_selected_pure_data.joblib")


In [3]:
%matplotlib inline

# import libraries
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample
from sklearn.metrics import precision_recall_curve
import bisect 
from scipy.stats import mstats
import matplotlib.ticker as plticker

# specifying constant parameters
random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)

# code for uploading pickled data
M_X_test_entFe = pickle.load(open("pure_data\M_X_test_entFe.pkl", 'rb'))
M_Y_test_entFe = pickle.load(open("pure_data\M_Y_test_entFe.pkl", 'rb'))
# M_X_test_entFe = pd.concat(M_X_test_entFe.values(), join='inner', ignore_index=True)
# M_Y_test_entFe = pd.concat(M_Y_test_entFe.values(), join='inner', ignore_index=True)

F_X_test_entFe = pickle.load(open("pure_data\F_X_test_entFe.pkl", 'rb'))
F_Y_test_entFe = pickle.load(open("pure_data\F_Y_test_entFe.pkl", 'rb'))
# F_X_test_entFe = pd.concat(F_X_test_entFe.values(), join='inner', ignore_index=True)
# F_Y_test_entFe = pd.concat(F_Y_test_entFe.values(), join='inner', ignore_index=True)

# select features
M_selected_features = ['Age', 'Relationship with family head','Respondent live in household',
                        'Known HIV status','Bought/sold sex in the past 12 months',
                        'Condom was used at last paid sex in past 12 months','LAg: recent/long term infection',
                        'CD4 category','on ART','ART initiated past 12 months those ever on ART',
                        'last sex partner relations past 12 months','Sick to work last three months',
                        'Ever attended school','Ever enrolled in school','Highest level of education',
                        'Highest grade at that school level', 'Work for payment in last 12 months',
                        'Ever married/lived together','Avoiding pregnancy','CFcumcision status',
                        'Age at fFst sex', 'Ever sought TB treatment','Alcohol drink frequency',
                        'Urban area indicator','Wealth quintile', 'Sexual intercourse in the past 12 months',
                        'Ever had sexual intercourse']

M_X_test_entFe = M_X_test_entFe[M_selected_features]

F_selected_features = ['Age', 'Relationship with family head', 'Respondent live in household',
       'Sick to work last three months', 'Ever attended school',
       'Ever enrolled in school', 'Highest level of education',
       'Highest grade at that school level',
       'Work for payment in last 12 months', 'Ever married/lived together',
       'Number of pregnancies',
       'Ever had a pregnancy that resulted in a live bFth',
       'No. of childen given bFth since 2012', 'Pregnacy status now',
       'Avoiding pregnancy', 'Age at fFst sex', 'Ever sought TB treatment',
       'Alcohol drink frequency', 'Urban area indicator', 'Known HIV status',
       'Wealth quintile', 'tested HIV at pregnancy',
       'Delivered 12 months preceeding survey',
       'Self-reported ARV status during pregnancy', 'Breastfed last child',
       'Sexual intercourse past 12 months', 'Ever had sexual intercourse',
       'Bought/sold sex in the past 12 months',
       'Condom was used at last paid sex in past 12 months',
       'last sex partner relations past 12 months',
       'LAg: recent/long term infection', 'CD4 category', 'On ART',
       'ART initiated past 12 months those ever on ART',
       'Whether ARVs detected']
  
F_X_test_entFe = F_X_test_entFe[F_selected_features]

M_model = load("xgb_selected\M_xgb_selected_pure_data.joblib")
F_model = load("xgb_selected\F_xgb_selected_pure_data.joblib")

In [ ]:
# import libraries
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load
from matplotlib import pyplot
import shap
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, precision_recall_curve, auc

# specifying constant parameters
random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)

# code for uploading pickled data
M_X_test_entFe = pickle.load(open("pure_data\M_X_test_entFe.pkl", 'rb'))
M_Y_test_entFe = pickle.load(open("pure_data\M_Y_test_entFe.pkl", 'rb'))
# M_X_test_entFe = pd.concat(M_X_test_entFe.values(), join='inner', ignore_index=True)
# M_Y_test_entFe = pd.concat(M_Y_test_entFe.values(), join='inner', ignore_index=True)

F_X_test_entFe = pickle.load(open("pure_data\F_X_test_entFe.pkl", 'rb'))
F_Y_test_entFe = pickle.load(open("pure_data\F_Y_test_entFe.pkl", 'rb'))
# F_X_test_entFe = pd.concat(F_X_test_entFe.values(), join='inner', ignore_index=True)
# F_Y_test_entFe = pd.concat(F_Y_test_entFe.values(), join='inner', ignore_index=True)

M_X_train_entFe = pickle.load(open("pure_data\M_X_train_entFe.pkl", 'rb'))
M_Y_train_entFe = pickle.load(open("pure_data\M_Y_train_entFe.pkl", 'rb'))
F_X_train_entFe = pickle.load(open("pure_data\F_X_train_entFe.pkl", 'rb'))
F_Y_train_entFe = pickle.load(open("pure_data\F_Y_train_entFe.pkl", 'rb'))

# M_X_train_entFe = pd.concat(M_X_train_entFe.values(), join='inner', ignore_index=True)
# M_Y_train_entFe = pd.concat(M_Y_train_entFe.values(), join='inner', ignore_index=True)
# F_X_train_entFe = pd.concat(F_X_train_entFe.values(), join='inner', ignore_index=True)
# F_Y_train_entFe = pd.concat(F_Y_train_entFe.values(), join='inner', ignore_index=True)
# select features
M_selected_features = ['Age', 'Relationship with family head','Respondent live in household',
                        'Known HIV status','Bought/sold sex in the past 12 months',
                        'Condom was used at last paid sex in past 12 months','LAg: recent/long term infection',
                        'CD4 category','on ART','ART initiated past 12 months those ever on ART',
                        'last sex partner relations past 12 months','Sick to work last three months',
                        'Ever attended school','Ever enrolled in school','Highest level of education',
                        'Highest grade at that school level', 'Work for payment in last 12 months',
                        'Ever married/lived together','Avoiding pregnancy','CFcumcision status',
                        'Age at fFst sex', 'Ever sought TB treatment','Alcohol drink frequency',
                        'Urban area indicator','Wealth quintile', 'Sexual intercourse in the past 12 months',
                        'Ever had sexual intercourse']


F_selected_features = ['Age', 'Relationship with family head', 'Respondent live in household',
       'Sick to work last three months', 'Ever attended school',
       'Ever enrolled in school', 'Highest level of education',
       'Highest grade at that school level',
       'Work for payment in last 12 months', 'Ever married/lived together',
       'Number of pregnancies',
       'Ever had a pregnancy that resulted in a live bFth',
       'No. of childen given bFth since 2012', 'Pregnacy status now',
       'Avoiding pregnancy', 'Age at fFst sex', 'Ever sought TB treatment',
       'Alcohol drink frequency', 'Urban area indicator', 'Known HIV status',
       'Wealth quintile', 'tested HIV at pregnancy',
       'Delivered 12 months preceeding survey',
       'Self-reported ARV status during pregnancy', 'Breastfed last child',
       'Sexual intercourse past 12 months', 'Ever had sexual intercourse',
       'Bought/sold sex in the past 12 months',
       'Condom was used at last paid sex in past 12 months',
       'last sex partner relations past 12 months',
       'LAg: recent/long term infection', 'CD4 category', 'On ART',
       'ART initiated past 12 months those ever on ART',
       'Whether ARVs detected']
 


M_X_train_selected = M_X_train_entFe[M_selected_features]
F_X_train_selected = F_X_train_entFe[F_selected_features]
M_X_test_selected = M_X_test_entFe[M_selected_features]
F_X_test_selected = F_X_test_entFe[F_selected_features]
     
M_xgb_selected = load("xgb_selected\M_xgb_selected_pure_data.joblib")
F_xgb_selected = load("xgb_selected\F_xgb_selected_pure_data.joblib")

M_best_index = M_xgb_selected.best_index_
F_best_index = F_xgb_selected.best_index_

M_best_index = M_xgb_selected.best_index_
F_best_index = F_xgb_selected.best_index_

M_Y_train_selected_pred = M_xgb_selected.predict(M_X_train_selected)
# M_Y_train_selected_pred_thresh = (M_xgb_selected.predict_proba(M_X_train_selected)[:,1] >= 0.3).astype(bool) # set threshold as 0.3
F_Y_train_selected_pred = F_xgb_selected.predict(F_X_train_selected)

print('Confusion matrix on the  M train dataset:', confusion_matrix(M_Y_train_entFe, M_Y_train_selected_pred))
print('Average F1 score on the  M train sample', "{:.1%}".format(M_xgb_selected.cv_results_['mean_test_f1'][M_best_index]))
print('CI F1 score on the M train sample', "{:.1%}".format(2*M_xgb_selected.cv_results_['std_test_f1'][M_best_index]))
print('Average Sensitivity on the M train sample', "{:.1%}".format(M_xgb_selected.cv_results_['mean_test_recall'][M_best_index]))
print('CI Sensitivity on the M train sample', "{:.1%}".format(2*M_xgb_selected.cv_results_['std_test_recall'][M_best_index]))
print('Average Positive Predictive Value on the M train sample', "{:.1%}".format(M_xgb_selected.cv_results_['mean_test_precision'][M_best_index]))
print('CI Positive Predictive Value on the M train sample', "{:.1%}".format(2*M_xgb_selected.cv_results_['std_test_precision'][M_best_index]))
print('Parameters', M_xgb_selected.best_estimator_)


print('Confusion matrix on the F train dataset:', confusion_matrix(F_Y_train_entFe, F_Y_train_selected_pred))
print('Average F1 score on the F train sample', "{:.1%}".format(F_xgb_selected.cv_results_['mean_test_f1'][F_best_index]))
print('CI F1 score on the F train sample', "{:.1%}".format(2*F_xgb_selected.cv_results_['std_test_f1'][F_best_index]))
print('Average Sensitivity on the F train sample', "{:.1%}".format(F_xgb_selected.cv_results_['mean_test_recall'][F_best_index]))
print('CI Sensitivity on the F train sample', "{:.1%}".format(2*F_xgb_selected.cv_results_['std_test_recall'][F_best_index]))
print('Average Positive Predictive Value on the F train sample', "{:.1%}".format(F_xgb_selected.cv_results_['mean_test_precision'][F_best_index]))
print('CI Positive Predictive Value on the F train sample', "{:.1%}".format(2*F_xgb_selected.cv_results_['std_test_precision'][F_best_index]))
print('Parameters', F_xgb_selected.best_estimator_)

from matplotlib import pyplot   
M_Y_test_selected_pred = M_xgb_selected.predict(M_X_test_selected)
F_Y_test_selected_pred = F_xgb_selected.predict(F_X_test_selected)



print('Confusion matrix on the M test dataset:', confusion_matrix(M_Y_test_entFe, M_Y_test_selected_pred))
print('F1 score on the M test dataset:', "{:.1%}".format(f1_score(M_Y_test_entFe, M_Y_test_selected_pred)))

print('Sensitivity on the M test dataset:', "{:.1%}".format(recall_score(M_Y_test_entFe, M_Y_test_selected_pred)))
print('Positive Predictive Value on the M test dataset:', "{:.1%}".format(precision_score(M_Y_test_entFe, M_Y_test_selected_pred)))


print('Confusion matrix on the F test dataset:', confusion_matrix(F_Y_test_entFe, F_Y_test_selected_pred))
print('F1 score on the F test dataset:', "{:.1%}".format(f1_score(F_Y_test_entFe, F_Y_test_selected_pred)))
print('Sensitivity on the F test dataset:', "{:.1%}".format(recall_score(F_Y_test_entFe, F_Y_test_selected_pred)))
print('Positive Predictive Value on the F test dataset:', "{:.1%}".format(precision_score(F_Y_test_entFe, F_Y_test_selected_pred)))




def pr_f1_curve(model, dx_test, dy_test):
     dy_test_predict_proba = model.predict_proba(dx_test)[:,1]
     dy_test_predict = model.predict(dx_test)
     ppv_curve, sensitivity_curve, _ = precision_recall_curve(dy_test, dy_test_predict_proba)
     F1_score, AUC_score = f1_score(dy_test, dy_test_predict), auc(sensitivity_curve, ppv_curve)
     print(': F1 score = %.2f - Area under Curve = %.2f' % (F1_score, AUC_score))
     prevalence = np.sum(dy_test)/len(dy_test)
     pyplot.plot([0, 1], [prevalence, prevalence], linestyle='--', label='Random Testing')
     pyplot.plot(sensitivity_curve, ppv_curve, marker='.', label='Algorithmic Testing')
     pyplot.plot(sensitivity_curve, 2*(sensitivity_curve*ppv_curve)/(sensitivity_curve+ppv_curve), marker='', label='F1')
     pyplot.xlabel('Sensitivity')
     pyplot.ylabel('Positive Predicted Value')
     pyplot.legend(bbox_to_anchor=(0.45, 0.75))
     pyplot.show()

pr_f1_curve(M_xgb_selected, M_X_test_selected, M_Y_test_entFe)
pr_f1_curve(F_xgb_selected, F_X_test_selected, F_Y_test_entFe)

# Explain model predictions using shapley value
M_explainer = shap.TreeExplainer(M_xgb_selected.best_estimator_)
M_shap_values = M_explainer.shap_values(M_X_train_selected)
F_explainer = shap.TreeExplainer(F_xgb_selected.best_estimator_)
F_shap_values = F_explainer.shap_values(F_X_train_selected)

# Plot summary_plot
M_shap = shap.summary_plot(M_shap_values, M_X_train_selected, max_display=32)
F_shap = shap.summary_plot(F_shap_values, F_X_train_selected, max_display=40)


In [ ]:
M_shap = shap.summary_plot(M_shap_values, M_X_train_selected, max_display=20)
F_shap = shap.summary_plot(F_shap_values, F_X_train_selected, max_display=25)
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt


plt.savefig('/home/dr/phia data/M_shap.png')
plt.savefig('/home/dr/phia data/F_shap.png')

In [ ]:
Confusion matrix on the train dataset: [[30658    26]
 [    3  2864]]
Average F1 score on the train sample 72.0%
CI F1 score on the train sample 1.6%
Average Sensitivity on the train sample 67.7%
CI Sensitivity on the train sample 3.8%
Average Positive Predictive Value on the train sample 76.9%
CI Positive Predictive Value on the train sample 3.1%
Parameters XGBClassifier(colsample_bytree=0.7, learning_rate=0.5, max_depth=10,
              min_split_loss=0.4, missing=nan, n_estimators=210, nthread=1,
              random_state=5, reg_alpha=0.6000000000000001, reg_lambda=1.0,
              scale_pos_weight=4)
Confusion matrix on the F train dataset: [[31261    15]
 [    2  4806]]
Average F1 score on the F train sample 92.2%
CI F1 score on the F train sample 1.7%
Average Sensitivity on the F train sample 88.7%
CI Sensitivity on the F train sample 3.8%
Average Positive Predictive Value on the F train sample 96.0%
CI Positive Predictive Value on the F train sample 1.4%
Parameters XGBClassifier(colsample_bytree=0.7, learning_rate=0.5, max_depth=10,
              min_split_loss=0.4, missing=nan, n_estimators=210, nthread=1,
              random_state=5, reg_alpha=0.6000000000000001, reg_lambda=1.0,
              scale_pos_weight=4)
Confusion matrix on the test dataset: [[7601  102]
 [ 171  514]]
F1 score on the test dataset: 79.0%
Sensitivity on the test dataset: 75.0%
Positive Predictive Value on the test dataset: 83.4%
Confusion matrix on the F test dataset: [[7811   37]
 [ 124 1049]]
F1 score on the F test dataset: 92.9%
Sensitivity on the F test dataset: 89.4%
Positive Predictive Value on the F test dataset: 96.6%
: F1 score = 0.79 - Area under Curve = 0.82

In [ ]:
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score
from xgboost import XGBClassifier
from matplotlib import pyplot
import shap
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, precision_recall_curve, auc


# specifying constant parameters
random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)
M_model = load("xgb_selected\M_xgb_selected_pure_data.joblib")
F_model = load("xgb_selected\F_xgb_selected_pure_data.joblib")
M_precision, M_recall, M_thresholds = precision_recall_curve(M_Y_test_entFe, M_model.predict_proba(M_X_test_selected)[:,1])

M_f1 = 2*M_recall*M_precision/(M_recall+M_precision)

F_precision, F_recall, F_thresholds = precision_recall_curve(F_Y_test_entFe, F_model.predict_proba(F_X_test_selected)[:,1])

F_f1 = 2*F_recall*F_precision/(F_recall+F_precision)


# plot the precision-recall curves

from itertools import cycle
# setup plot details
colors = cycle(['navy', 'turquoise', 'darkorange', 'cornflowerblue', 'teal'])

plt.figure(figsize=(7, 8))
f_scores = np.linspace(0.2, 0.8, num=4)
lines = []
labels = []
for f_score in f_scores:
    x = np.linspace(0.01, 1)
    y = f_score * x / (2 * x - f_score)
    l, = plt.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.2)
    plt.annotate('f1={0:0.1f}'.format(f_score), xy=(0.9, y[45] + 0.02))

lines.append(l)
labels.append('iso-f1 curves')
l, = plt.plot(recall["M_recall"], precision["M_precision"], color='gold', lw=2)
lines.append(l)
labels.append('micro-average Precision-recall (area = {0:0.2f})'
              ''.format(average_precision["micro"]))

for i, color in zip(range(n_classes), colors):
    l, = plt.plot(recall[i], precision[i], color=color, lw=2)
    lines.append(l)
    labels.append('Precision-recall for class {0} (area = {1:0.2f})'
                  ''.format(i, average_precision[i]))

fig = plt.gcf()
fig.subplots_adjust(bottom=0.25)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Extension of Precision-Recall curve to multi-class')
plt.legend(lines, labels, loc=(0, -.38), prop=dict(size=14))


plt.show()

In [ ]:
# plot the precision-recall curves
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
# setup plot details
colors = cycle(['navy', 'turquoise', 'darkorange', 'cornflowerblue', 'teal'])

plt.figure(figsize=(7, 8))
f_scores = np.linspace(0.2, 0.8, num=4)
lines = []
labels = []
for f_score in f_scores:
    x = np.linspace(0.01, 1)
    y = f_score * x / (2 * x - f_score)
    l, = plt.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.2)
    plt.annotate('f1={0:0.1f}'.format(f_score), xy=(0.9, y[45] + 0.02))

lines.append(l)
labels.append('iso-f1 curves')
l, = plt.plot(recall["micro"], precision["micro"], color='gold', lw=2)
lines.append(l)
labels.append('micro-average Precision-recall (area = {0:0.2f})'
              ''.format(average_precision["micro"]))

for i, color in zip(range(n_classes), colors):
    l, = plt.plot(recall[i], precision[i], color=color, lw=2)
    lines.append(l)
    labels.append('Precision-recall for class {0} (area = {1:0.2f})'
                  ''.format(i, average_precision[i]))

fig = plt.gcf()
fig.subplots_adjust(bottom=0.25)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Extension of Precision-Recall curve to multi-class')
plt.legend(lines, labels, loc=(0, -.38), prop=dict(size=14))


plt.show()




import numpy as np
from sklearn.metrics import precision_recall_curve
y_true = np.array([0, 0, 1, 1])
y_scores = np.array([0.1, 0.4, 0.35, 0.8])
precision, recall, thresholds = precision_recall_curve(
...     y_true, y_scores)
precision
array([0.66666667, 0.5       , 1.        , 1.        ])
>>> recall
array([1. , 0.5, 0.5, 0. ])
>>> thresholds
array([0.35, 0.4 , 0.8 ])

In [ ]:
M_index = np.argmax(M_precision[(M_recall>=0.949999999) & (M_recall >= 0.9500000001)])

print('Males')
print('Best f1: ', M_f1[M_index])
print('Best precision: ', M_precision[M_index])
print('Best threshold: ', M_thresholds[M_index])

F_index = np.argmax(F_precision[(F_recall>=0.949999999) & (F_recall >= 0.9500000001)])

print('Females')
print('Best f1: ', F_f1[F_index])
print('Best precision: ', F_precision[F_index])
print('Best threshold: ', F_thresholds[F_index])

In [ ]:
from sklearn.metrics import confusion_matrix
M_Y_test_pred_thresh = (M_model.predict_proba(M_X_test_selected)[:,1] >= 0.0018904804).astype(bool)

print('Males')
confusion_matrix(M_Y_test_entFe, M_Y_test_pred_thresh)

F_Y_test_pred_thresh = (F_model.predict_proba(F_X_test_selected)[:,1] >= 0.007321233).astype(bool)

print('Females')
confusion_matrix(F_Y_test_entFe, F_Y_test_pred_thresh)

In [ ]:
M_Y_test_pred_thresh = (M_model.predict_proba(M_X_test_selected)[:,1] >= 0.95).astype(bool)

print('Males')
confusion_matrix(M_Y_test_entFe, M_Y_test_pred_thresh)


F_Y_test_pred_thresh = (F_model.predict_proba(F_X_test_selected)[:,1] >= 0.95).astype(bool)

print('Females')
confusion_matrix(F_Y_test_entFe, F_Y_test_pred_thresh)

In [ ]:
print('Best F1-Score: ', np.max(M_f1))
print('Best precision: ', M_precision[np.argmax(M_f1)])
print('Best recall: ', M_recall[np.argmax(M_f1)])
print('Best threshold: ', M_thresholds[np.argmax(M_f1)])

print('Best F1-Score: ', np.max(F_f1))
print('Best precision: ', F_precision[np.argmax(F_f1)])
print('Best recall: ', F_recall[np.argmax(F_f1)])
print('Best threshold: ', F_thresholds[np.argmax(F_f1)])

In [ ]:
# import libraries
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# specifying constant parameters
random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)
# code for uploading pickled data

# code for uploading pickled data
# M_X_test_entFe = pickle.load(open("pure_data\M_X_test_entFe.pkl", 'rb'))
# M_Y_test_entFe = pickle.load(open("pure_data\M_Y_test_entFe.pkl", 'rb'))
M_X_train_entFe = pickle.load(open("pure_data\M_X_train_entFe.pkl", 'rb'))
M_Y_train_entFe = pickle.load(open("pure_data\M_Y_train_entFe.pkl", 'rb'))
F_X_train_entFe = pickle.load(open("pure_data\F_X_train_entFe.pkl", 'rb'))
F_Y_train_entFe = pickle.load(open("pure_data\F_Y_train_entFe.pkl", 'rb'))

# stratified k-fold and model initialization
# stratified k-fold and model initialization
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)

M_model = load("xgb_selected\M_xgb_selected_pure_data.joblib")
F_model = load("xgb_selected\F_xgb_selected_pure_data.joblib")

M_sfs = SFS(M_model.best_estimator_, k_features=20, forward=True, floating=False, scoring='f1', verbose=1, cv=0, n_jobs=-1)
F_sfs = SFS(F_model.best_estimator_, k_features=25, forward=True, floating=True, scoring='f1', verbose=1, cv=0, n_jobs=-1)

M_sfs.fit(M_X_train_entFe, M_Y_train_entFe)
F_sfs.fit(F_X_train_entFe, F_Y_train_entFe)

dump(M_sfs, 'M_sfs.joblib')
dump(F_sfs, 'F_sfs.joblib')

In [ ]:
%matplotlib inline

from joblib import load
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as plticker
from matplotlib.patches import PathPatch
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

# import file
M_sfs_pure = load("M_sfs.joblib")
F_sfs_pure = load("F_sfs.joblib")

M_dict_list = list(M_sfs_pure.get_metric_dict().values())
F_dict_list = list(F_sfs_pure.get_metric_dict().values())

M_score = []
F_score = []

for index in range(len(M_dict_list)):
    M_score.append(M_dict_list[index]['avg_score'])
    F_score.append(F_dict_list[index]['avg_score'])

def adjust_box_widths(g, fac):
    """
    Adjust the withs of a seaborn-generated boxplot.
    """

    # iterating through Axes instances
    for ax in g.axes:

        # iterating through axes artists:
        for c in ax.get_children():

            # searching for PathPatches
            if isinstance(c, PathPatch):
                # getting current width of box:
                p = c.get_path()
                verts = p.vertices
                verts_sub = verts[:-1]
                xmin = np.min(verts_sub[:, 0])
                xmax = np.max(verts_sub[:, 0])
                xmid = 0.5*(xmin+xmax)
                xhalf = 0.5*(xmax - xmin)

                # setting new width of box
                xmin_new = xmid-fac*xhalf
                xmax_new = xmid+fac*xhalf
                verts_sub[verts_sub[:, 0] == xmin, 0] = xmin_new
                verts_sub[verts_sub[:, 0] == xmax, 0] = xmax_new

                # setting new width of median line
                for l in ax.lines:
                    if np.all(l.get_xdata() == [xmin, xmax]):
                        l.set_xdata([xmin_new, xmax_new])


y_loc = plticker.MultipleLocator(base=0.1)
fig, (ax1, ax2, ax3, ax4) = plt.subplots(2, 2, figsize=(22, 11))
plt.subplots_adjust(wspace=0)
sns.set(palette='deep')
sns.set_context("notebook", font_scale=2, rc={"lines.linewidth": 4})
sns.lineplot(x=list(range(1, 31)), y=M_score, palette="Blues", ax = ax1)
sns.lineplot(x=list(range(1, 31)), y=F_score, palette="Blues", ax = ax2)
ax1.title.set_text('Males')
ax2.title.set_text('Females')
ax1.set_xlabel('Number of variables', fontsize=16)
ax2.set_xlabel('Number of variables', fontsize=16)
ax1.set_ylabel('f1 score')
ax2.set_ylabel('')
ax1.set(ylim=(0, 1))
ax2.set(ylim=(0, 1))
ax1.yaxis.set_major_locator(y_loc)
ax2.yaxis.set_major_locator(y_loc)
ax1.grid(b=True, which='major', axis='y')
ax2.grid(b=True, which='major', axis='y')
plt.setp(ax2.get_yticklabels(), visible=False)
fig.suptitle('Sequential floating forward selection', fontsize=24)

adjust_box_widths(fig, 0.8)
plt.savefig('sffs.png', bbox_inches='tight')

In [ ]:
# M_sfs_pure.subsets_
F_sfs_pure.subsets_

In [12]:
M_sfs_pure = load("M_sfs.joblib")
F_sfs_pure = load("F_sfs.joblib")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
fig1 = plot_sfs(M_sfs_pure.get_metric_dict(),
                kind='std_dev',
                figsize=(22, 11))

plt.ylim([0.3, 1])
plt.ylabel('f1 score')
plt.title('Sequential Forward Selection')
plt.grid()
plt.show

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
fig1 = plot_sfs(F_sfs_pure.get_metric_dict(),
                kind='std_dev',
                figsize=(22, 11))

plt.ylim([0.85, 1])
plt.ylabel('f1 score')
plt.title('Sequential Forward Selection')
plt.grid()
plt.show

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

plot_sfs(M_sfs_pure.get_metric_dict(),
                kind='std_dev',
                figsize=(10, 6))

plt.ylim([0.3, 1])
plt.xlabel('Number of variables', fontsize=20)
plt.ylabel('f1 score', fontsize=20)
plt.title('SFFS - males', fontsize=20)
plt.grid()
plt.savefig('M_sfsfinal.jpg')

plot_sfs(F_sfs_pure.get_metric_dict(),
                kind='std_dev',
                figsize=(10, 6))

plt.ylim([0.85, 1])
plt.xlabel('Number of variables', fontsize=20)
plt.ylabel('f1 score', fontsize=20)
plt.title('SFFS - females', fontsize=20)
plt.grid()
plt.savefig('F_sfsfinal.jpg')

In [ ]:
sns.lineplot(x=list(range(1, 30)), y=M_score, palette="Blues", ax = ax1)
sns.lineplot(x=list(range(1, 30)), y=F_score, palette="Blues", ax = ax1)

In [ ]:
################################################################

In [ ]:
###############################################################

In [ ]:
# import libraries
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score
from xgboost import XGBClassifier
from matplotlib import pyplot
import matplotlib.pyplot as plt
import shap
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, precision_recall_curve, auc


# specifying constant parameters
random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)

# code for uploading pickled data
M_X_train_entFe = pickle.load(open("pure_data\M_X_train_entFe.pkl", 'rb'))
M_Y_train_entFe = pickle.load(open("pure_data\M_Y_train_entFe.pkl", 'rb'))
F_X_train_entFe = pickle.load(open("pure_data\F_X_train_entFe.pkl", 'rb'))
F_Y_train_entFe = pickle.load(open("pure_data\F_Y_train_entFe.pkl", 'rb'))

# M_X_train_entFe = pd.concat(M_X_train_entFe.values(), join='inner', ignore_index=True)
# M_Y_train_entFe = pd.concat(M_Y_train_entFe.values(), join='inner', ignore_index=True)
# F_X_train_entFe = pd.concat(F_X_train_entFe.values(), join='inner', ignore_index=True)
# F_Y_train_entFe = pd.concat(F_Y_train_entFe.values(), join='inner', ignore_index=True)
# select features
M_selected_features = ['Age',
   'Relationship with family head',
   'Ever enrolled in school',
   'Highest level of education',
   'Highest grade at that school level',
   'Work for payment in last 12 months',
   'Avoiding pregnancy',
   'CFcumcision status',
   'Age at fFst sex',
   'Ever sought TB treatment',
   'Alcohol drink frequency',
   'Urban area indicator',
   'Wealth quintile']  
M_X_train_selected = M_X_train_entFe[M_selected_features]

F_selected_features = ['Age',
   'Relationship with family head',
   'Highest level of education',
   'Highest grade at that school level',
   'Work for payment in last 12 months',
   'Ever married/lived together',
   'Number of pregnancies',
   'No. of childen given bFth since 2012',
   'Avoiding pregnancy',
   'Age at fFst sex',
   'Ever sought TB treatment',
   'Known HIV status',
   'Wealth quintile']    
     
F_X_train_selected = F_X_train_entFe[F_selected_features]

#================================================================================================================================
# parameters space, models definition and random grid search
#================================================================================================================================

# stratified k-fold and model initialization
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)
scoring = ['f1', 'recall', 'precision']

# XGBoost space
params_xgb = {'learning_rate': np.linspace(0, 1, 11),
              'min_split_loss': np.linspace(0, 1, 6),
              'max_depth': np.linspace(2, 10, 5, dtype=int),
              'min_child_weight': np.linspace(1, 20, 20, dtype=int),
              'colsample_bytree': np.linspace(0.5, 1, 6),
              'reg_alpha': np.linspace(0, 1, 11),
              'scale_pos_weight': np.linspace(4, 50, 24, dtype=int),
              'n_estimators': np.linspace(10, 450, 12, dtype=int),
              'reg_lambda': np.linspace(0, 10, 11),
             }

M_xgb_selected_15 = RandomizedSearchCV(estimator=XGBClassifier(booster='gbtree',
                                                                     objective='binary:logistic',
                                                                     random_state=random_seed,
                                                                     nthread=1,
                                                                     verbosity=1),
                                             param_distributions=params_xgb,
                                             n_iter=250,
                                             scoring=scoring,
                                             cv=kfold,
                                             n_jobs=-1,
                                             random_state=random_seed,
                                             verbose=2,
                                             refit='f1')

F_xgb_selected_12 = RandomizedSearchCV(estimator=XGBClassifier(booster='gbtree',
                                                                     objective='binary:logistic',
                                                                     random_state=random_seed,
                                                                     nthread=1,
                                                                     verbosity=1),
                                             param_distributions=params_xgb,
                                             n_iter=250,
                                             scoring=scoring,
                                             cv=kfold,
                                             n_jobs=-1,
                                             random_state=random_seed,
                                             verbose=2,
                                             refit='f1')

M_xgb_selected_15.fit(M_X_train_selected, M_Y_train_entFe)
dump(M_xgb_selected_15, "xgb_selected\M_xgb_selected_15_data.joblib")

F_xgb_selected_12.fit(F_X_train_selected, F_Y_train_entFe)
dump(F_xgb_selected_12, "xgb_selected\F_xgb_selected_12_data.joblib")

In [21]:
%matplotlib inline

# import libraries
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample
from sklearn.metrics import precision_recall_curve
import bisect 
from scipy.stats import mstats
import matplotlib.ticker as plticker

# specifying constant parameters
random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)

# code for uploading pickled data
M_X_test_entFe = pickle.load(open("pure_data\M_X_test_entFe.pkl", 'rb'))
M_Y_test_entFe = pickle.load(open("pure_data\M_Y_test_entFe.pkl", 'rb'))
# M_X_test_entFe = pd.concat(M_X_test_entFe.values(), join='inner', ignore_index=True)
# M_Y_test_entFe = pd.concat(M_Y_test_entFe.values(), join='inner', ignore_index=True)

F_X_test_entFe = pickle.load(open("pure_data\F_X_test_entFe.pkl", 'rb'))
F_Y_test_entFe = pickle.load(open("pure_data\F_Y_test_entFe.pkl", 'rb'))
# F_X_test_entFe = pd.concat(F_X_test_entFe.values(), join='inner', ignore_index=True)
# F_Y_test_entFe = pd.concat(F_Y_test_entFe.values(), join='inner', ignore_index=True)

# select features
M_selected_features = ['Age',
   'Relationship with family head',
   'Ever enrolled in school',
   'Highest level of education',
   'Highest grade at that school level',
   'Work for payment in last 12 months',
   'Avoiding pregnancy',
   'CFcumcision status',
   'Age at fFst sex',
   'Ever sought TB treatment',
   'Alcohol drink frequency',
   'Urban area indicator',
   'Wealth quintile']  
M_X_train_selected = M_X_train_entFe[M_selected_features]


F_selected_features = ['Age',
   'Relationship with family head',
   'Highest level of education',
   'Highest grade at that school level',
   'Work for payment in last 12 months',
   'Ever married/lived together',
   'Number of pregnancies',
   'No. of childen given bFth since 2012',
   'Avoiding pregnancy',
   'Age at fFst sex',
   'Ever sought TB treatment',
   'Known HIV status',
   'Wealth quintile']  
F_X_test_entFe = F_X_test_entFe[F_selected_features]

M_model_15 = load("xgb_selected\M_xgb_selected_15_data.joblib")
F_model_12 = load("xgb_selected\F_xgb_selected_12_data.joblib")

In [ ]:
# import libraries
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load
from matplotlib import pyplot
%matplotlib inline
import matplotlib.pyplot as plt
import shap
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, precision_recall_curve, auc

# specifying constant parameters
random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)

# code for uploading pickled data
M_X_test_entFe = pickle.load(open("pure_data\M_X_test_entFe.pkl", 'rb'))
M_Y_test_entFe = pickle.load(open("pure_data\M_Y_test_entFe.pkl", 'rb'))
# M_X_test_entFe = pd.concat(M_X_test_entFe.values(), join='inner', ignore_index=True)
# M_Y_test_entFe = pd.concat(M_Y_test_entFe.values(), join='inner', ignore_index=True)

F_X_test_entFe = pickle.load(open("pure_data\F_X_test_entFe.pkl", 'rb'))
F_Y_test_entFe = pickle.load(open("pure_data\F_Y_test_entFe.pkl", 'rb'))
# F_X_test_entFe = pd.concat(F_X_test_entFe.values(), join='inner', ignore_index=True)
# F_Y_test_entFe = pd.concat(F_Y_test_entFe.values(), join='inner', ignore_index=True)

M_X_train_entFe = pickle.load(open("pure_data\M_X_train_entFe.pkl", 'rb'))
M_Y_train_entFe = pickle.load(open("pure_data\M_Y_train_entFe.pkl", 'rb'))
F_X_train_entFe = pickle.load(open("pure_data\F_X_train_entFe.pkl", 'rb'))
F_Y_train_entFe = pickle.load(open("pure_data\F_Y_train_entFe.pkl", 'rb'))

# M_X_train_entFe = pd.concat(M_X_train_entFe.values(), join='inner', ignore_index=True)
# M_Y_train_entFe = pd.concat(M_Y_train_entFe.values(), join='inner', ignore_index=True)
# F_X_train_entFe = pd.concat(F_X_train_entFe.values(), join='inner', ignore_index=True)
# F_Y_train_entFe = pd.concat(F_Y_train_entFe.values(), join='inner', ignore_index=True)
# select features
M_selected_features = ['Age',
   'Relationship with family head',
   'Ever enrolled in school',
   'Highest level of education',
   'Highest grade at that school level',
   'Work for payment in last 12 months',
   'Avoiding pregnancy',
   'CFcumcision status',
   'Age at fFst sex',
   'Ever sought TB treatment',
   'Alcohol drink frequency',
   'Urban area indicator',
   'Wealth quintile']  
M_X_train_selected = M_X_train_entFe[M_selected_features]

F_selected_features = ['Age',
   'Relationship with family head',
   'Highest level of education',
   'Highest grade at that school level',
   'Work for payment in last 12 months',
   'Ever married/lived together',
   'Number of pregnancies',
   'No. of childen given bFth since 2012',
   'Avoiding pregnancy',
   'Age at fFst sex',
   'Ever sought TB treatment',
   'Known HIV status',
   'Wealth quintile'] 


M_X_train_selected = M_X_train_entFe[M_selected_features]
F_X_train_selected = F_X_train_entFe[F_selected_features]
M_X_test_selected = M_X_test_entFe[M_selected_features]
F_X_test_selected = F_X_test_entFe[F_selected_features]
     
M_xgb_selected = load("xgb_selected\M_xgb_selected_15_data.joblib")
F_xgb_selected = load("xgb_selected\F_xgb_selected_12_data.joblib")

M_best_index = M_xgb_selected.best_index_
F_best_index = F_xgb_selected.best_index_

M_Y_train_selected_pred = M_xgb_selected.predict(M_X_train_selected)
# M_Y_train_selected_pred_thresh = (M_xgb_selected.predict_proba(M_X_train_selected)[:,1] >= 0.3).astype(bool) # set threshold as 0.3
F_Y_train_selected_pred = F_xgb_selected.predict(F_X_train_selected)

print('Confusion matrix on the train dataset:', confusion_matrix(M_Y_train_entFe, M_Y_train_selected_pred))
print('Average F1 score on the train sample', "{:.1%}".format(M_xgb_selected.cv_results_['mean_test_f1'][M_best_index]))
print('CI F1 score on the train sample', "{:.1%}".format(2*M_xgb_selected.cv_results_['std_test_f1'][M_best_index]))
print('Average Sensitivity on the train sample', "{:.1%}".format(M_xgb_selected.cv_results_['mean_test_recall'][M_best_index]))
print('CI Sensitivity on the train sample', "{:.1%}".format(2*M_xgb_selected.cv_results_['std_test_recall'][M_best_index]))
print('Average Positive Predictive Value on the train sample', "{:.1%}".format(M_xgb_selected.cv_results_['mean_test_precision'][M_best_index]))
print('CI Positive Predictive Value on the train sample', "{:.1%}".format(2*M_xgb_selected.cv_results_['std_test_precision'][M_best_index]))
print('Parameters', M_xgb_selected.best_estimator_)

print('Confusion matrix on the F train dataset:', confusion_matrix(F_Y_train_entFe, F_Y_train_selected_pred))
print('Average F1 score on the F train sample', "{:.1%}".format(F_xgb_selected.cv_results_['mean_test_f1'][F_best_index]))
print('CI F1 score on the F train sample', "{:.1%}".format(2*F_xgb_selected.cv_results_['std_test_f1'][F_best_index]))
print('Average Sensitivity on the F train sample', "{:.1%}".format(F_xgb_selected.cv_results_['mean_test_recall'][F_best_index]))
print('CI Sensitivity on the F train sample', "{:.1%}".format(2*F_xgb_selected.cv_results_['std_test_recall'][F_best_index]))
print('Average Positive Predictive Value on the F train sample', "{:.1%}".format(F_xgb_selected.cv_results_['mean_test_precision'][F_best_index]))
print('CI Positive Predictive Value on the F train sample', "{:.1%}".format(2*F_xgb_selected.cv_results_['std_test_precision'][F_best_index]))
print('Parameters', F_xgb_selected.best_estimator_)
     
M_Y_test_selected_pred = M_xgb_selected.predict(M_X_test_selected)
F_Y_test_selected_pred = F_xgb_selected.predict(F_X_test_selected)

print('Confusion matrix on the test dataset:', confusion_matrix(M_Y_test_entFe, M_Y_test_selected_pred))
print('F1 score on the test dataset:', "{:.1%}".format(f1_score(M_Y_test_entFe, M_Y_test_selected_pred)))
print('Sensitivity on the test dataset:', "{:.1%}".format(recall_score(M_Y_test_entFe, M_Y_test_selected_pred)))
print('Positive Predictive Value on the test dataset:', "{:.1%}".format(precision_score(M_Y_test_entFe, M_Y_test_selected_pred)))

print('Confusion matrix on the F test dataset:', confusion_matrix(F_Y_test_entFe, F_Y_test_selected_pred))
print('F1 score on the F test dataset:', "{:.1%}".format(f1_score(F_Y_test_entFe, F_Y_test_selected_pred)))
print('Sensitivity on the F test dataset:', "{:.1%}".format(recall_score(F_Y_test_entFe, F_Y_test_selected_pred)))
print('Positive Predictive Value on the F test dataset:', "{:.1%}".format(precision_score(F_Y_test_entFe, F_Y_test_selected_pred)))


def pr_f1_curve(model, dx_test, dy_test):
     dy_test_predict_proba = model.predict_proba(dx_test)[:,1]
     dy_test_predict = model.predict(dx_test)
     ppv_curve, sensitivity_curve, _ = precision_recall_curve(dy_test, dy_test_predict_proba)
     F1_score, AUC_score = f1_score(dy_test, dy_test_predict), auc(sensitivity_curve, ppv_curve)
     print(': F1 score = %.2f - Area under Curve = %.2f' % (F1_score, AUC_score))
     prevalence = np.sum(dy_test)/len(dy_test)
     pyplot.plot([0, 1], [prevalence, prevalence], linestyle='--', label='Random Testing')
     pyplot.plot(sensitivity_curve, ppv_curve, marker='.', label='Algorithmic Testing')
     pyplot.plot(sensitivity_curve, 2*(sensitivity_curve*ppv_curve)/(sensitivity_curve+ppv_curve), marker='', label='F1')
     pyplot.xlabel('Sensitivity')
     pyplot.ylabel('Positive Predicted Value')
     pyplot.legend(bbox_to_anchor=(0.45, 0.75))
     pyplot.show()

pr_f1_curve(M_xgb_selected, M_X_test_selected, M_Y_test_entFe)
pr_f1_curve(F_xgb_selected, F_X_test_selected, F_Y_test_entFe)

# Explain model predictions using shapley value
M_explainer = shap.TreeExplainer(M_xgb_selected.best_estimator_)
M_shap_values = M_explainer.shap_values(M_X_train_selected)
F_explainer = shap.TreeExplainer(F_xgb_selected.best_estimator_)
F_shap_values = F_explainer.shap_values(F_X_train_selected)

# Plot summary_plot
M_shap = shap.summary_plot(M_shap_values, M_X_train_selected, max_display=20)
F_shap = shap.summary_plot(F_shap_values, F_X_train_selected, max_display=25)
plt.savefig('M_shapfinal.jpg')
plt.savefig('F_shapfinal.jpg')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
M_shap = shap.summary_plot(M_shap_values, M_X_train_selected, max_display=20)
F_shap = shap.summary_plot(F_shap_values, F_X_train_selected, max_display=25)
plt.savefig('M_shap.jpg')
plt.savefig('F_shap.jpg')


In [ ]:
M_model_15 = load("xgb_selected\M_xgb_selected_15_data.joblib")
F_model_12 = load("xgb_selected\F_xgb_selected_12_data.joblib")
M_precision, M_recall, M_thresholds = precision_recall_curve(M_Y_test_entFe, M_model_15.predict_proba(M_X_test_selected)[:,1])

M_f1 = 2*M_recall*M_precision/(M_recall+M_precision)

F_precision, F_recall, F_thresholds = precision_recall_curve(F_Y_test_entFe, F_model_12.predict_proba(F_X_test_selected)[:,1])

F_f1 = 2*F_recall*F_precision/(F_recall+F_precision)

In [ ]:
M_index = np.argmax(M_precision[(M_recall>=0.949999999) & (M_recall >= 0.9500000001)])

print('Males')
print('Best f1: ', M_f1[M_index])
print('Best precision: ', M_precision[M_index])
print('Best threshold: ', M_thresholds[M_index])

F_index = np.argmax(F_precision[(F_recall>=0.899999999) & (F_recall >= 0.9000000001)])

print('Females')
print('Best f1: ', F_f1[F_index])
print('Best precision: ', F_precision[F_index])
print('Best threshold: ', F_thresholds[F_index])

In [ ]:
from sklearn.metrics import confusion_matrix
# M_Y_test_pred_thresh = (M_model_15.predict_proba(M_X_test_selected)[:,1] >= 0.002331057).astype(bool)

# print('Males')
# confusion_matrix(M_Y_test_entFe, M_Y_test_pred_thresh)

F_Y_test_pred_thresh = (F_model_12.predict_proba(F_X_test_selected)[:,1] >= 0.12806101).astype(bool)

print('Females')
confusion_matrix(F_Y_test_entFe, F_Y_test_pred_thresh)

In [ ]:
# M_Y_test_pred_thresh = (M_model_15.predict_proba(M_X_test_selected)[:,1] >= 0.90).astype(bool)
F_Y_test_pred_thresh = (F_model_12.predict_proba(F_X_test_selected)[:,1] >= 0.90).astype(bool)
# print('Males')
# confusion_matrix(M_Y_test_entFe, M_Y_test_pred_thresh)
print('Females')
confusion_matrix(F_Y_test_entFe, F_Y_test_pred_thresh)

In [ ]:
print('Best F1-Score: ', np.max(M_f1))
print('Best precision: ', M_precision[np.argmax(M_f1)])
print('Best recall: ', M_recall[np.argmax(M_f1)])
print('Best threshold: ', M_thresholds[np.argmax(M_f1)])

print('Best F1-Score: ', np.max(F_f1))
print('Best precision: ', F_precision[np.argmax(F_f1)])
print('Best recall: ', F_recall[np.argmax(F_f1)])
print('Best threshold: ', F_thresholds[np.argmax(F_f1)])

In [ ]:
%who